In [1]:
# ===== OpenFDA + Hetionet MERGE (Colab one-cell; GraphML-safe) =====
# 1) SET THESE PATHS:
OPENFDA_GRAPHML = "/home/ubuntu/myproject/venv/drug_data_kg_openfda_edges_no_drop.graphml"        # <-- change to your file
HETIONET_GRAPHML = "/home/ubuntu/myproject/venv/hetionet_normalized.graphml"      # <-- change to your file
OUTDIR = "/home/ubuntu/myproject/venv/merged_ofda_hetionet"            # <-- change if you like

# (Optional) dataset tags to stamp into nodes/edges
OPENFDA_VERSION = "openfda_YYYYMMDD"
HETIONET_VERSION = "hetionet_vX.Y"

# 2) Collapse policy (safe defaults)
COLLAPSE_DRUGS = True
COLLAPSE_DISEASES = False        # turn on only if you truly have curated names/IDs
COLLAPSE_PHENOTYPES = False      # turn on only if you truly have curated names/IDs

# Drug collapse thresholds/guards
DRUG_TIER3_MIN_SCORE = 0.97      # fuzzy unique match minimum
DRUG_TIER3_MIN_MARGIN = 0.02     # winner - runner-up margin
PRODUCT_LIKE_MAX_COMPONENTS = 1  # >1 components => product-like (don't collapse)

# Disease/phenotype collapse thresholds (stricter)
DIS_TIER3_MIN_SCORE = 0.98
DIS_TIER3_MIN_MARGIN = 0.03

# Attribute keys in your GraphMLs
NODE_TYPE_KEY = "node_type"
EDGE_REL_KEY  = "relation"
NODE_NAME_KEYS = ["node_name", "name", "label"]   # tried in this order

# 3) Outputs: choose formats (PKL is always written; others are optional & GraphML-safe)
WRITE_GRAPHML = False     # set True if you need GraphML (slower; attributes JSON-stringified)
WRITE_GEXF    = False      # GEXF is generally faster and also string-safe here

# --- (Optional) Mount Google Drive if your files live there ---
# from google.colab import drive
# drive.mount('/content/drive')

# --- Install deps (quiet) ---
!pip -q install networkx lxml

# -------------------- PIPELINE (no edits needed below) --------------------
import os, re, json, pickle, csv, random
from collections import defaultdict, Counter
from typing import Dict, Any, Tuple, List
import networkx as nx
from difflib import SequenceMatcher

random.seed(42)
os.makedirs(OUTDIR, exist_ok=True)

def load_graphml_as_multidigraph(path: str) -> nx.MultiDiGraph:
    G = nx.read_graphml(path)
    if not G.is_directed():
        G = G.to_directed()
    if not isinstance(G, (nx.MultiDiGraph, nx.MultiGraph)):
        G = nx.MultiDiGraph(G)
    return G

def first_nonempty(d, keys=NODE_NAME_KEYS):
    for k in keys:
        if k in d and d[k] is not None and str(d[k]).strip():
            return str(d[k])
    return ""

# ----- Normalization helpers -----
FORMULATION_TOKENS = {
    "tablet","tablets","tab","tabs","capsule","capsules","cap","caps","cream","ointment","gel","lozenge","suspension",
    "solution","spray","lotion","patch","injection","syrup","elixir","drops","chewable","extended","release","er","xr","xl","dr","cr",
    "immediate","delayed","topical","oral","nasal","ophthalmic","otic","rectal","vaginal","sublingual",
    "mg","mcg","g","gram","grams","ml","%","iu","spf","wt","vol","per","dose","strength","unit","units"
}
DELIMS_FOR_MIXTURES = [",","/","+"," and "," with "]

def norm_name_basic(s: str) -> str:
    s = s.upper().strip()
    s = re.sub(r"[^A-Z0-9 ,/+]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def tokenize_clean(s: str) -> List[str]:
    s = norm_name_basic(s)
    tokens = [t for t in re.split(r"[ ,/+]", s) if t]
    tokens = [t for t in tokens if not t.isdigit()]
    tokens = [t for t in tokens if t.lower() not in FORMULATION_TOKENS]
    return tokens

def jaccard(a: List[str], b: List[str]) -> float:
    A, B = set(a), set(b)
    return 0.0 if not A or not B else len(A & B) / len(A | B)

def fuzzy_ratio(a: str, b: str) -> float:
    return SequenceMatcher(None, norm_name_basic(a), norm_name_basic(b)).ratio()

def combined_score(name_a: str, name_b: str) -> float:
    # blend sequence + token Jaccard (weights tuned conservatively)
    t = jaccard(tokenize_clean(name_a), tokenize_clean(name_b))
    f = fuzzy_ratio(name_a, name_b)
    return 0.6 * f + 0.4 * t

def is_mixture_like(name: str) -> bool:
    s = norm_name_basic(name)
    return any(d in s for d in DELIMS_FOR_MIXTURES)

# ----- OpenFDA normalization -----
def normalize_openfda(G: nx.MultiDiGraph) -> nx.MultiDiGraph:
    out = nx.MultiDiGraph()
    for n, d in G.nodes(data=True):
        dd = dict(d)
        dd["node_source"] = "openfda"
        dd["node_name"] = first_nonempty(dd)
        out.add_node(n, **dd)

    kept = dropped = flipped = 0
    for u, v, ed in G.edges(data=True):
        rel = str(ed.get(EDGE_REL_KEY, "unknown"))
        su = str(G.nodes[u].get(NODE_TYPE_KEY, "unknown"))
        sv = str(G.nodes[v].get(NODE_TYPE_KEY, "unknown"))
        ed2 = dict(ed)
        ed2["edge_source"] = "openfda"
        ed2["display_relation"] = rel

        if rel == "metaedge":
            dropped += 1
            continue

        if rel == "drug_drug":
            ed2["relation"] = "drug_has_component"
            out.add_edge(u, v, **ed2); kept += 1; continue

        if rel == "drug_effect":
            if su == "drug" and sv == "effect/phenotype":
                ed2["relation"] = "drug_effect"
                out.add_edge(u, v, **ed2); kept += 1; continue
            if su == "drug" and sv == "exposure":
                ed2["relation"] = "drug_exposure"
                out.add_edge(u, v, **ed2); kept += 1; continue
            if su == "effect/phenotype" and sv == "drug":
                ed2["relation"] = "drug_effect"
                ed2.setdefault("attrs", {})
                if not isinstance(ed2["attrs"], dict): ed2["attrs"] = {"_raw_attrs": ed2["attrs"]}
                ed2["attrs"]["method"] = "flipped"
                out.add_edge(v, u, **ed2); kept += 1; flipped += 1; continue
            if su == "effect/phenotype" and sv in ("effect/phenotype","exposure"):
                ed2["relation"] = "drug_effect"
                ed2.setdefault("attrs", {})
                if not isinstance(ed2["attrs"], dict): ed2["attrs"] = {"_raw_attrs": ed2["attrs"]}
                ed2["attrs"]["noncanonical"] = True
                ed2["attrs"]["variant"] = "effect_to_effect" if sv == "effect/phenotype" else "effect_to_exposure"
                out.add_edge(u, v, **ed2); kept += 1; continue
            ed2["relation"] = "drug_effect"
            out.add_edge(u, v, **ed2); kept += 1; continue

        if rel in ("indication","contraindication","drug_disease"):
            ed2["relation"] = rel
            out.add_edge(u, v, **ed2); kept += 1; continue

        ed2["relation"] = rel
        out.add_edge(u, v, **ed2); kept += 1

    print(f"[normalize_openfda] kept={kept:,} dropped(metaedge)={dropped:,} flipped={flipped:,}")
    return out

# ----- Hetionet normalization -----
def normalize_hetionet(G: nx.MultiDiGraph) -> nx.MultiDiGraph:
    out = nx.MultiDiGraph()
    kept = dropped = renamed = 0
    for n, d in G.nodes(data=True):
        dd = dict(d)
        dd["node_source"] = "hetionet"
        dd["node_name"] = first_nonempty(dd)
        out.add_node(n, **dd)
    for u, v, ed in G.edges(data=True):
        rel = str(ed.get(EDGE_REL_KEY, "unknown"))
        if rel == "metaedge":
            dropped += 1; continue
        ed2 = dict(ed)
        ed2["edge_source"] = "hetionet"
        if rel == "drug_drug":
            ed2["relation"] = "drug_drug_similarity"
            ed2["display_relation"] = ed.get("display_relation", "CrC")
            renamed += 1
        else:
            ed2["relation"] = rel
            ed2.setdefault("display_relation", rel)
        out.add_edge(u, v, **ed2); kept += 1
    print(f"[normalize_hetionet] kept={kept:,} dropped(metaedge)={dropped:,} renamed(drug_drug->similarity)={renamed:,}")
    return out

# ----- Name indices for collapse -----
def build_name_index(G: nx.MultiDiGraph, node_type: str) -> Dict[str, Dict[str, Any]]:
    idx = {}
    for n, d in G.nodes(data=True):
        if str(d.get(NODE_TYPE_KEY, "")) != node_type: continue
        nm = first_nonempty(d)
        if nm: idx[n] = {"name": nm, "tokens": tokenize_clean(nm)}
    return idx

def product_like_by_structure(ofdaG: nx.MultiDiGraph, node_id: str) -> bool:
    # if it has > PRODUCT_LIKE_MAX_COMPONENTS outgoing composition edges, treat as product-like
    comps = 0
    if ofdaG.has_node(node_id):
        for _, v, ed in ofdaG.out_edges(node_id, data=True):
            if ed.get("relation") == "drug_has_component":
                comps += 1
                if comps > PRODUCT_LIKE_MAX_COMPONENTS: return True
    return False

def build_drug_collapse_map(ofdaG: nx.MultiDiGraph, hetG: nx.MultiDiGraph) -> Dict[str, str]:
    ofda_idx = build_name_index(ofdaG, "drug")
    het_idx  = build_name_index(hetG,  "drug")
    het_by_exact = defaultdict(list)
    for hid, rec in het_idx.items():
        het_by_exact[norm_name_basic(rec["name"])].append(hid)

    mapping = {}
    considered = tier2 = tier3 = skipped = 0

    for oid, rec in ofda_idx.items():
        considered += 1
        oname = rec["name"]

        # guard: product-like?
        if is_mixture_like(oname) or product_like_by_structure(ofdaG, oid):
            skipped += 1; continue

        # TIER 2: exact normalized name match
        ex = norm_name_basic(oname)
        if ex in het_by_exact and len(het_by_exact[ex]) == 1:
            mapping[oid] = het_by_exact[ex][0]; tier2 += 1; continue

        # TIER 3: fuzzy unique match
        best_id, best_score, second = None, -1.0, -1.0
        for hid, hrec in het_idx.items():
            sc = combined_score(oname, hrec["name"])
            if sc > best_score:
                second = best_score; best_score = sc; best_id = hid
            elif sc > second:
                second = sc
        if best_id is not None and best_score >= DRUG_TIER3_MIN_SCORE and (best_score - second) >= DRUG_TIER3_MIN_MARGIN:
            mapping[oid] = best_id; tier3 += 1
        else:
            skipped += 1

    print(f"[collapse:drugs] considered={considered:,} exact={tier2:,} fuzzy={tier3:,} skipped={skipped:,}")
    return mapping

def build_other_collapse_map(ofdaG: nx.MultiDiGraph, hetG: nx.MultiDiGraph, node_type: str,
                             min_score=0.98, min_margin=0.03) -> Dict[str, str]:
    ofda_idx = build_name_index(ofdaG, node_type)
    het_idx  = build_name_index(hetG,  node_type)
    het_by_exact = defaultdict(list)
    for hid, rec in het_idx.items():
        het_by_exact[norm_name_basic(rec["name"])].append(hid)
    mapping = {}
    considered = tier2 = tier3 = skipped = 0
    for oid, rec in ofda_idx.items():
        considered += 1
        oname = rec["name"]
        ex = norm_name_basic(oname)
        if ex in het_by_exact and len(het_by_exact[ex]) == 1:
            mapping[oid] = het_by_exact[ex][0]; tier2 += 1; continue
        # fuzzy
        best_id, best_score, second = None, -1.0, -1.0
        for hid, hrec in het_idx.items():
            sc = combined_score(oname, hrec["name"])
            if sc > best_score:
                second = best_score; best_score = sc; best_id = hid
            elif sc > second:
                second = sc
        if best_id is not None and best_score >= min_score and (best_score - second) >= min_margin:
            mapping[oid] = best_id; tier3 += 1
        else:
            skipped += 1
    print(f"[collapse:{node_type}] considered={considered:,} exact={tier2:,} fuzzy={tier3:,} skipped={skipped:,}")
    return mapping

# ----- Canonicalization & edge accumulation -----
def add_or_merge_node(canon: Dict[str, Dict[str,Any]], gid: str, base: Dict[str, Any]):
    if gid not in canon:
        canon[gid] = dict(base)
        return
    N = canon[gid]
    # node_source merge
    def to_list(x):
        if x is None: return []
        if isinstance(x, list): return x
        return [x]
    srcs = set(to_list(N.get("node_source"))) | set(to_list(base.get("node_source")))
    N["node_source"] = sorted(srcs) if srcs else None
    # alias names & ids
    N.setdefault("alias_names", [])
    if base.get("node_name") and base["node_name"] not in N["alias_names"]:
        N["alias_names"].append(base["node_name"])
    N.setdefault("aliases", [])
    for a in to_list(base.get("aliases")):
        if a not in N["aliases"]:
            N["aliases"].append(a)
    # attrs.sources merge
    N.setdefault("attrs", {})
    N["attrs"].setdefault("sources", {})
    battrs = {k:v for k,v in base.get("attrs", {}).items()}
    src_label = base.get("node_source", "unknown")
    if isinstance(src_label, list): src_label = "_".join(src_label)
    N["attrs"]["sources"][src_label] = battrs

def coalesce_edge_payload(cur: Dict[str, Any], newe: Dict[str, Any]) -> Dict[str, Any]:
    def to_list(x):
        if x is None: return []
        if isinstance(x, list): return x
        return [x]
    cur["edge_source"] = sorted(set(to_list(cur.get("edge_source")) + to_list(newe.get("edge_source"))))
    cur["display_relation"] = sorted(set(to_list(cur.get("display_relation")) + to_list(newe.get("display_relation"))))
    cur.setdefault("attrs", {})
    cur["attrs"].setdefault("sources", {})
    ns = newe.get("edge_source", "unknown")
    if isinstance(ns, list): ns = "_".join(ns)
    cur["attrs"]["sources"][ns] = newe.get("attrs", {})
    return cur

# ----- GraphML/GEXF-safe copier (JSON-stringify non-scalars) -----
def _graphml_safe_val(v):
    if v is None: return ""
    if isinstance(v, (str, int, float, bool)): return v
    try:
        return json.dumps(v, ensure_ascii=False)
    except Exception:
        return str(v)

def make_graphml_safe(G):
    G2 = nx.MultiDiGraph()
    for n, d in G.nodes(data=True):
        d2 = {k: _graphml_safe_val(v) for k, v in d.items()}
        G2.add_node(n, **d2)
    for u, v, ed in G.edges(data=True):
        ed2 = {k: _graphml_safe_val(val) for k, val in ed.items()}
        G2.add_edge(u, v, **ed2)
    return G2

# ----- Main runner -----
def run_merge():
    # Load
    G_ofda_raw = load_graphml_as_multidigraph(OPENFDA_GRAPHML)
    G_het_raw  = load_graphml_as_multidigraph(HETIONET_GRAPHML)
    print(f"[load] OpenFDA:  nodes={G_ofda_raw.number_of_nodes():,}, edges={G_ofda_raw.number_of_edges():,}")
    print(f"[load] Hetionet: nodes={G_het_raw.number_of_nodes():,}, edges={G_het_raw.number_of_edges():,}")

    # Normalize
    G_ofda = normalize_openfda(G_ofda_raw)
    G_het  = normalize_hetionet(G_het_raw)

    # Collapse maps
    collapse_map = {}
    if COLLAPSE_DRUGS:
        collapse_map.update(build_drug_collapse_map(G_ofda, G_het))
    if COLLAPSE_DISEASES:
        collapse_map.update(build_other_collapse_map(G_ofda, G_het, "disease", DIS_TIER3_MIN_SCORE, DIS_TIER3_MIN_MARGIN))
    if COLLAPSE_PHENOTYPES:
        collapse_map.update(build_other_collapse_map(G_ofda, G_het, "effect/phenotype", DIS_TIER3_MIN_SCORE, DIS_TIER3_MIN_MARGIN))

    # Canonical node table
    canon_nodes: Dict[str, Dict[str,Any]] = {}
    def mk_node(nid: str, d: Dict[str,Any], src_prefix: str) -> Dict[str,Any]:
        rec = {
            "gid": f"{src_prefix}:{nid}",
            "node_id": str(nid),
            "node_type": d.get(NODE_TYPE_KEY, "unknown"),
            "node_name": first_nonempty(d),
            "node_source": src_prefix,
            "attrs": {k:v for k,v in d.items() if k not in (NODE_TYPE_KEY, "node_source", "node_name")}
        }
        if src_prefix == "openfda" and OPENFDA_VERSION:
            rec["version"] = OPENFDA_VERSION
        if src_prefix == "hetionet" and HETIONET_VERSION:
            rec["version"] = HETIONET_VERSION
        return rec

    # Add Hetionet canonical nodes
    for n, d in G_het.nodes(data=True):
        canon_nodes[f"hetionet:{n}"] = mk_node(n, d, "hetionet")

    # Add OpenFDA nodes (collapse when mapped)
    collapsed = []
    kept_openfda = []
    for n, d in G_ofda.nodes(data=True):
        if n in collapse_map:
            hid = collapse_map[n]
            canon_gid = f"hetionet:{hid}"
            base = mk_node(n, d, "openfda")
            base["aliases"] = [base["node_id"]]
            add_or_merge_node(canon_nodes, canon_gid, base)
            collapsed.append((n, hid, d.get(NODE_TYPE_KEY)))
        else:
            canon_nodes[f"openfda:{n}"] = mk_node(n, d, "openfda")
            kept_openfda.append((n, d.get(NODE_TYPE_KEY)))

    # Edge accumulation (with rewiring & dedupe)
    edge_bucket: Dict[Tuple[str,str,str], Dict[str,Any]] = {}

    def acc_edge(src_native: str, dst_native: str, ed: Dict[str,Any], is_openfda: bool):
        if is_openfda:
            sg = f"hetionet:{collapse_map[src_native]}" if src_native in collapse_map else f"openfda:{src_native}"
            dg = f"hetionet:{collapse_map[dst_native]}" if dst_native in collapse_map else f"openfda:{dst_native}"
        else:
            sg, dg = f"hetionet:{src_native}", f"hetionet:{dst_native}"
        rel = ed.get("relation")
        key = (sg, rel, dg)
        payload = {
            "relation": rel,
            "edge_source": ed.get("edge_source", "openfda" if is_openfda else "hetionet"),
            "display_relation": ed.get("display_relation", rel),
            "attrs": {k:v for k,v in ed.items() if k not in ("relation","edge_source","display_relation")}
        }
        if is_openfda and OPENFDA_VERSION: payload["version"] = OPENFDA_VERSION
        if (not is_openfda) and HETIONET_VERSION: payload["version"] = HETIONET_VERSION

        if key in edge_bucket:
            edge_bucket[key] = coalesce_edge_payload(edge_bucket[key], payload)
        else:
            edge_bucket[key] = payload

    for u, v, ed in G_ofda.edges(data=True):
        acc_edge(u, v, ed, is_openfda=True)
    for u, v, ed in G_het.edges(data=True):
        acc_edge(u, v, ed, is_openfda=False)

    # Build final graph
    G_final = nx.MultiDiGraph()
    for gid, nd in canon_nodes.items():
        G_final.add_node(gid, **nd)
    for (su, rel, sv), pd in edge_bucket.items():
        G_final.add_edge(su, sv, **pd)

    # Write always: PKL, collapse map, JSON report
    out_pkl     = os.path.join(OUTDIR, "combined_openfda_hetionet.pkl")
    out_map_csv = os.path.join(OUTDIR, "collapsed_openfda_to_hetionet.csv")
    out_report  = os.path.join(OUTDIR, "merge_report.json")
    with open(out_pkl, "wb") as f:
        pickle.dump(G_final, f)

    with open(out_map_csv, "w", newline="", encoding="utf-8") as f:
        w = csv.writer(f); w.writerow(["openfda_node_id","hetionet_node_id","node_type"])
        for oid, hid, t in collapsed: w.writerow([oid, hid, t])

    def count_nodes_by_type(G):
        c = Counter(str(d.get(NODE_TYPE_KEY, "unknown")) for _, d in G.nodes(data=True))
        return dict(c)
    def count_edges_by_rel(G):
        c = Counter(str(d.get("relation","unknown")) for _, _, d in G.edges(data=True))
        return dict(c)

    rep = {
        "inputs": {
            "openfda_graphml": OPENFDA_GRAPHML,
            "hetionet_graphml": HETIONET_GRAPHML,
            "openfda_version": OPENFDA_VERSION,
            "hetionet_version": HETIONET_VERSION
        },
        "policy": {
            "collapse_drugs": COLLAPSE_DRUGS,
            "collapse_diseases": COLLAPSE_DISEASES,
            "collapse_phenotypes": COLLAPSE_PHENOTYPES,
            "drug_tier3_min_score": DRUG_TIER3_MIN_SCORE,
            "drug_tier3_min_margin": DRUG_TIER3_MIN_MARGIN,
            "disease_tier3_min_score": DIS_TIER3_MIN_SCORE,
            "disease_tier3_min_margin": DIS_TIER3_MIN_MARGIN
        },
        "stats": {
            "hetionet_norm": {"nodes": G_het.number_of_nodes(), "edges": G_het.number_of_edges()},
            "openfda_norm": {"nodes": G_ofda.number_of_nodes(), "edges": G_ofda.number_of_edges()},
            "collapsed_pairs": len(collapsed),
            "final": {"nodes": G_final.number_of_nodes(), "edges": G_final.number_of_edges()},
            "final_nodes_by_type": count_nodes_by_type(G_final),
            "final_edges_by_relation": count_edges_by_rel(G_final)
        }
    }
    with open(out_report, "w", encoding="utf-8") as f:
        json.dump(rep, f, ensure_ascii=False, indent=2)

    # Optional: GraphML/GEXF (GraphML-safe)
    wrote_graphml = wrote_gexf = False
    if WRITE_GRAPHML or WRITE_GEXF:
        G_safe = make_graphml_safe(G_final)
        if WRITE_GRAPHML:
            out_graphml = os.path.join(OUTDIR, "combined_openfda_hetionet.graphml")
            nx.write_graphml(G_safe, out_graphml)
            wrote_graphml = True
        if WRITE_GEXF:
            out_gexf = os.path.join(OUTDIR, "combined_openfda_hetionet.gexf")
            nx.write_gexf(G_safe, out_gexf)
            wrote_gexf = True

    print("\n=== DONE ===")
    print(f"• Combined PKL:      {out_pkl}")
    if wrote_graphml: print(f"• Combined GraphML:  {out_graphml}")
    if wrote_gexf:    print(f"• Combined GEXF:     {out_gexf}")
    print(f"• Collapse map CSV:  {out_map_csv}  (rows={len(collapsed)})")
    print(f"• Merge report JSON: {out_report}")
    print("\nFinal summary:")
    print(json.dumps(rep["stats"], indent=2))

run_merge()


[load] OpenFDA:  nodes=100,495, edges=160,229
[load] Hetionet: nodes=47,033, edges=1,379,933
[normalize_openfda] kept=160,144 dropped(metaedge)=85 flipped=135
[normalize_hetionet] kept=1,379,932 dropped(metaedge)=1 renamed(drug_drug->similarity)=6,486
[collapse:drugs] considered=55,430 exact=769 fuzzy=0 skipped=54,661

=== DONE ===
• Combined PKL:      /home/ubuntu/myproject/venv/merged_ofda_hetionet/combined_openfda_hetionet.pkl
• Collapse map CSV:  /home/ubuntu/myproject/venv/merged_ofda_hetionet/collapsed_openfda_to_hetionet.csv  (rows=769)
• Merge report JSON: /home/ubuntu/myproject/venv/merged_ofda_hetionet/merge_report.json

Final summary:
{
  "hetionet_norm": {
    "nodes": 47033,
    "edges": 1379932
  },
  "openfda_norm": {
    "nodes": 100495,
    "edges": 160144
  },
  "collapsed_pairs": 769,
  "final": {
    "nodes": 146759,
    "edges": 1401041
  },
  "final_nodes_by_type": {
    "anatomy": 402,
    "biological_process": 11381,
    "cellular_component": 1391,
    "drug": 5

In [2]:
# === Pickle (NetworkX) → R-GCN (NeighborLoader + AMP + batched inference) ===
# Fixes:
#  - DistMult scoring (relation-aware) in train + eval
#  - Typed negatives sampled *within the batch* (local), avoiding drop
#  - zero_division=0 to silence undefined metric warnings

import os, gc, random, pickle, gzip
from collections import defaultdict

import numpy as np
import torch
import torch.nn.functional as F
from torch import nn
from torch.cuda.amp import autocast, GradScaler
from torch_geometric.nn import RGCNConv
from torch_geometric.data import Data
from torch_geometric.loader import NeighborLoader
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score
import networkx as nx

# ----------------------------
# 0) Repro + Device
# ----------------------------
SEED = 42
random.seed(SEED); np.random.seed(SEED)
torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)
os.environ.setdefault("PYTORCH_CUDA_ALLOC_CONF", "expandable_segments:True")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gc.collect()
if device.type == "cuda":
    torch.cuda.empty_cache()
print("Using device:", device)

# ----------------------------
# 1) Load PICKLE (NetworkX) + normalize attrs
# ----------------------------
# Accepts a pickle produced via: pickle.dump(G, open(path,'wb')) where G is a NetworkX graph
pickle_path = "/home/ubuntu/myproject/venv/merged_ofda_hetionet/combined_openfda_hetionet.pkl"  # <- change if needed
assert os.path.exists(pickle_path), f"Path does not exist: {pickle_path}"
print(f"📥 Loading NetworkX graph from pickle: {pickle_path}")

open_fn = gzip.open if pickle_path.endswith(".gz") else open
with open_fn(pickle_path, "rb") as f:
    G0 = pickle.load(f)

# Ensure MultiDiGraph container
if isinstance(G0, (nx.MultiDiGraph, nx.MultiGraph, nx.Graph, nx.DiGraph)):
    G = nx.MultiDiGraph()
    G.add_nodes_from(G0.nodes(data=True))
    if isinstance(G0, (nx.MultiDiGraph, nx.MultiGraph)):
        for u, v, k, d in G0.edges(keys=True, data=True):
            G.add_edge(u, v, key=k, **(d or {}))
    else:
        for u, v, d in G0.edges(data=True):
            G.add_edge(u, v, **(d or {}))
else:
    raise TypeError(f"Pickle did not contain a NetworkX graph, got {type(G0)}")

# Normalize expected attributes
for n, data in G.nodes(data=True):
    if "node_type" not in data:
        data["node_type"] = data.get("type", data.get("category", data.get("kind", "unknown")))
for _, _, attr in G.edges(data=True):
    if "relation" not in attr:
        attr["relation"] = attr.get("display_relation", attr.get("relationship", attr.get("type", attr.get("label", "unknown"))))

print(f"✅ Loaded graph with {len(G.nodes)} nodes and {len(G.edges)} edges.")

# ----------------------------
# 2) Typed remap + splits
# ----------------------------
node_type_map = defaultdict(list)
node_id_map = {}
for nid, data in G.nodes(data=True):
    ntype = data.get("node_type", "unknown")
    idx = len(node_type_map[ntype])
    node_type_map[ntype].append(nid)
    node_id_map[nid] = (ntype, idx)

edge_type_map = defaultdict(list)  # (src_type, rel, dst_type) -> [(src_local, dst_local)]
for s, t, attr in G.edges(data=True):
    if s not in node_id_map or t not in node_id_map:
        continue
    rel = attr.get("relation", "unknown")
    s_t, s_i = node_id_map[s]
    t_t, t_i = node_id_map[t]
    edge_type_map[(s_t, rel, t_t)].append((s_i, t_i))

print(f"✅ Node types: {list(node_type_map.keys())}")
print(f"✅ Total edge types: {len(edge_type_map)}")

# Global ids per (type, local)
type_offsets, global_id_map = {}, {}
offset = 0
for ntype, nodes in node_type_map.items():
    type_offsets[ntype] = offset
    for i in range(len(nodes)):
        global_id_map[(ntype, i)] = offset + i
    offset += len(nodes)
num_nodes = offset

edge_type_keys = list(edge_type_map.keys())  # rel_id -> (src_type, rel, dst_type)
rel2id = {k: i for i, k in enumerate(edge_type_keys)}
num_relations = len(edge_type_keys)

all_src, all_dst, all_rel = [], [], []
edge_type_splits = {}

for etype, pairs in edge_type_map.items():
    s_type, _, t_type = etype
    if len(pairs) < 5:  # keep only relations with minimum support
        continue
    perm = torch.randperm(len(pairs))
    n_tr = int(0.8 * len(pairs))
    n_va = int(0.1 * len(pairs))
    tr = [pairs[i] for i in perm[:n_tr]]
    va = [pairs[i] for i in perm[n_tr:n_tr + n_va]]
    te = [pairs[i] for i in perm[n_tr + n_va:]]

    r = rel2id[etype]
    for s_l, t_l in tr:
        all_src.append(type_offsets[s_type] + s_l)
        all_dst.append(type_offsets[t_type] + t_l)
        all_rel.append(r)

    edge_type_splits[etype] = {
        "val": torch.tensor(
            [[type_offsets[s_type] + s for s, _ in va],
             [type_offsets[t_type] + t for _, t in va]], dtype=torch.long),
        "test": torch.tensor(
            [[type_offsets[s_type] + s for s, _ in te],
             [type_offsets[t_type] + t for _, t in te]], dtype=torch.long),
    }

edge_index_cpu = torch.tensor([all_src, all_dst], dtype=torch.long)
edge_type_cpu = torch.tensor(all_rel, dtype=torch.long)
print(f"Homogeneous nodes: {num_nodes}, edges used for training: {edge_index_cpu.size(1)}")

# For quick type lookup by GLOBAL id (used for local typed negatives)
type_names = sorted(node_type_map.keys())
type2id = {t: i for i, t in enumerate(type_names)}
global_node_type_id = torch.empty(num_nodes, dtype=torch.long)
for tname, nodes in node_type_map.items():
    t_id = type2id[tname]
    base = type_offsets[tname]
    n = len(nodes)
    global_node_type_id[base:base + n] = t_id

# Existing edges per relation (LOCAL to each (src_type,rel,dst_type))
existing_edges_per_rel = {etype: set(pairs) for etype, pairs in edge_type_map.items()}
def relid_to_etype(rel_id: int):
    return edge_type_keys[rel_id]

# ----------------------------
# 3) PyG Data + NeighborLoader
# ----------------------------
data_pyg = Data(num_nodes=num_nodes, edge_index=edge_index_cpu, edge_type=edge_type_cpu)

NUM_NEIGHBORS = [15, 10]
NBATCH_SIZE   = 4096
NUM_WORKERS   = 2  # quiets the OS warning

train_src_unique = torch.unique(edge_index_cpu[0])
train_loader = NeighborLoader(
    data_pyg,
    num_neighbors=NUM_NEIGHBORS,
    batch_size=NBATCH_SIZE,
    input_nodes=train_src_unique,
    shuffle=True,
    num_workers=NUM_WORKERS
)

# ----------------------------
# 4) Model (DistMult) + helpers
# ----------------------------
class RGCN(nn.Module):
    def __init__(self, num_nodes, num_relations, emb_dim=64, hidden_dim=128, out_dim=32, num_bases=30):
        super().__init__()
        self.emb = nn.Embedding(num_nodes, emb_dim)
        self.conv1 = RGCNConv(emb_dim, hidden_dim, num_relations=num_relations, num_bases=num_bases)
        self.conv2 = RGCNConv(hidden_dim, out_dim, num_relations=num_relations, num_bases=num_bases)
        self.rel_emb = nn.Embedding(num_relations, out_dim)

        nn.init.xavier_uniform_(self.emb.weight)
        nn.init.xavier_uniform_(self.rel_emb.weight)

    def forward(self, n_id, edge_index, edge_type):
        # n_id: global ids ordered as local nodes in the subgraph
        x = self.emb(n_id)
        x = self.conv1(x, edge_index, edge_type).relu_()
        x = self.conv2(x, edge_index, edge_type)
        return x  # local node embeddings

    def score(self, h, r, t):
        # DistMult (logits)
        return (h * r * t).sum(dim=-1)

def distmult_sigmoid_scores(z, rel_emb, eidx, rel_ids):
    h = z[eidx[0]]; t = z[eidx[1]]
    r = rel_emb(rel_ids)
    return (h * r * t).sum(dim=-1).sigmoid()

# ----------------------------
# 5) Local (in-batch) typed negatives
# ----------------------------
def build_local_type_pools(n_id_device):
    """Map node_type_id -> list of LOCAL indices present in this batch."""
    g_types = global_node_type_id.to(n_id_device.device, non_blocking=True)[n_id_device]
    pools = defaultdict(list)
    for local_idx, t in enumerate(g_types.tolist()):
        pools[t].append(local_idx)
    for k in list(pools.keys()):
        pools[k] = torch.tensor(pools[k], dtype=torch.long, device=n_id_device.device)
    return pools

def sample_neg_local(pos_local_eidx, rel_ids_local, n_id, pools, k_neg=1):
    """
    Typed negatives *within* the sampled subgraph.
    pos_local_eidx: (2,P) local indices
    rel_ids_local:  (P,)   relation ids (global rel ids)
    pools: dict[type_id] -> local node index tensor present in batch
    """
    if pos_local_eidx.numel() == 0:  # empty
        return torch.empty((2,0), dtype=torch.long, device=pos_local_eidx.device)

    P = pos_local_eidx.size(1)
    neg_src, neg_dst = [], []
    for i in range(P):
        s_l = int(pos_local_eidx[0, i])
        d_l = int(pos_local_eidx[1, i])
        r_id = int(rel_ids_local[i])
        s_type, _, d_type = relid_to_etype(r_id)
        s_tid = type2id[s_type]; d_tid = type2id[d_type]

        # pick from local pools — if missing (rare), fall back to uniform local sampling
        s_pool = pools.get(s_tid, None)
        d_pool = pools.get(d_tid, None)
        if s_pool is None or s_pool.numel() == 0:
            s_pool = torch.arange(n_id.size(0), device=n_id.device)
        if d_pool is None or d_pool.numel() == 0:
            d_pool = torch.arange(n_id.size(0), device=n_id.device)

        for _ in range(max(1, k_neg)):
            if random.random() < 0.5:
                # corrupt head
                new_s_l = s_pool[torch.randint(0, s_pool.numel(), (1,))].item()
                neg_src.append(new_s_l); neg_dst.append(d_l)
            else:
                # corrupt tail
                new_d_l = d_pool[torch.randint(0, d_pool.numel(), (1,))].item()
                neg_src.append(s_l); neg_dst.append(new_d_l)

    if len(neg_src) == 0:
        return torch.empty((2,0), dtype=torch.long, device=n_id.device)
    return torch.stack([torch.tensor(neg_src, device=n_id.device),
                        torch.tensor(neg_dst, device=n_id.device)], dim=0)

# ----------------------------
# 6) Train with NeighborLoader + AMP
# ----------------------------
EPOCHS = 30
LR     = 1e-3
K_NEG  = 1

model = RGCN(num_nodes, num_relations, emb_dim=64, hidden_dim=128, out_dim=32, num_bases=30).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=1e-5)
scaler = GradScaler(enabled=(device.type == "cuda"))

# Global training edges (for picking positives per batch)
src_glob_all = edge_index_cpu[0].to(device)
dst_glob_all = edge_index_cpu[1].to(device)
rels_all     = edge_type_cpu.to(device)

print("\n🔁 Training with NeighborLoader mini-batches + AMP ...")
for epoch in range(1, EPOCHS+1):
    model.train()
    epoch_loss = 0.0
    seen_pos = 0

    for batch in train_loader:
        batch = batch.to(device, non_blocking=True)
        n_id = batch.n_id  # global ids in this subgraph (device tensor)

        # global->local mapping
        global2local = -torch.ones(num_nodes, dtype=torch.long, device=device)
        global2local[n_id] = torch.arange(n_id.size(0), device=device)

        # positives inside batch
        in_src = global2local[src_glob_all]
        in_dst = global2local[dst_glob_all]
        mask = (in_src >= 0) & (in_dst >= 0)
        pos_idx = torch.nonzero(mask, as_tuple=False).view(-1)
        if pos_idx.numel() == 0:
            del batch, n_id, global2local, in_src, in_dst, mask, pos_idx
            if device.type == "cuda": torch.cuda.empty_cache()
            continue

        MAX_POS_PER_BATCH = 8192
        if pos_idx.numel() > MAX_POS_PER_BATCH:
            pos_idx = pos_idx[torch.randperm(pos_idx.numel(), device=device)[:MAX_POS_PER_BATCH]]

        pos_local = torch.stack([in_src[pos_idx], in_dst[pos_idx]], dim=0)
        rel_ids_local = rels_all[pos_idx]

        # local typed pools for negatives
        pools = build_local_type_pools(n_id)

        optimizer.zero_grad(set_to_none=True)
        with autocast(enabled=(device.type == "cuda")):
            z = model(n_id, batch.edge_index, batch.edge_type)

            # DistMult positives
            pos_h = z[pos_local[0]]; pos_t = z[pos_local[1]]
            pos_r = model.rel_emb(rel_ids_local)
            pos_logits = model.score(pos_h, pos_r, pos_t)

            # local typed negatives
            neg_local = sample_neg_local(pos_local, rel_ids_local, n_id, pools, k_neg=K_NEG)
            if neg_local.size(1) > 0:
                neg_h = z[neg_local[0]]; neg_t = z[neg_local[1]]
                # repeat relation ids for negs
                neg_r = model.rel_emb(rel_ids_local.repeat_interleave(K_NEG))
                neg_logits = model.score(neg_h, neg_r, neg_t)
                loss = (F.binary_cross_entropy_with_logits(pos_logits, torch.ones_like(pos_logits)) +
                        F.binary_cross_entropy_with_logits(neg_logits, torch.zeros_like(neg_logits)))
            else:
                loss = F.binary_cross_entropy_with_logits(pos_logits, torch.ones_like(pos_logits))

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        epoch_loss += float(loss.detach().cpu())
        seen_pos += pos_local.size(1)

        # cleanup
        del batch, n_id, global2local, in_src, in_dst, mask, pos_idx
        del pos_local, rel_ids_local, pools, z, pos_h, pos_t, pos_r, pos_logits
        if 'neg_local' in locals(): del neg_local, neg_h, neg_t, neg_r, neg_logits
        if device.type == "cuda": torch.cuda.empty_cache()

    print(f"Epoch {epoch:02d} | Loss: {epoch_loss:.4f} | Seen pos edges: {seen_pos}")

# ----------------------------
# 7) Batched inference (DistMult-ready embeddings)
# ----------------------------
print("\n🔎 Computing full-node embeddings (batched inference) ...")
model.eval()
OUT_DIM = 32
z_all = torch.empty((num_nodes, OUT_DIM), dtype=torch.float32, device=device)

INFER_BATCH = 16384
infer_loader = NeighborLoader(
    Data(num_nodes=num_nodes, edge_index=edge_index_cpu, edge_type=edge_type_cpu),
    num_neighbors=[-1, -1],
    batch_size=INFER_BATCH,
    input_nodes=torch.arange(num_nodes),
    shuffle=False,
    num_workers=NUM_WORKERS
)
with torch.no_grad():
    for batch in infer_loader:
        batch = batch.to(device, non_blocking=True)
        n_id = batch.n_id
        z_batch = model(n_id, batch.edge_index, batch.edge_type)
        z_all[n_id] = z_batch
        del batch, z_batch, n_id
        if device.type == "cuda": torch.cuda.empty_cache()

# ----------------------------
# 8) Evaluation (per relation + overall) — DistMult + zero_division=0
# ----------------------------
print("\n📊 Final evaluation per edge type (filtered negatives, DistMult):")

def eval_relation(z_all, etype, splits, k_neg=1):
    rel_id = rel2id[etype]
    pos_e = splits["test"]
    if pos_e.numel() == 0: return None
    N = pos_e.size(1)

    # Build typed negatives globally (tails only for simplicity)
    src_g = pos_e[0].to(device); dst_g = pos_e[1].to(device)
    rel_ids = torch.full((N,), rel_id, dtype=torch.long, device=device)
    s_type, _, d_type = etype
    d_pool_global = torch.arange(type_offsets[d_type],
                                 type_offsets[d_type] + len(node_type_map[d_type]),
                                 device=device)
    neg_src = src_g.repeat_interleave(k_neg)
    neg_dst = d_pool_global[torch.randint(0, d_pool_global.numel(), (N * k_neg,), device=device)]

    # Scores
    pos_scores = distmult_sigmoid_scores(z_all, model.rel_emb, pos_e.to(device), rel_ids)
    neg_scores = distmult_sigmoid_scores(z_all, model.rel_emb,
                                         torch.stack([neg_src, neg_dst]),
                                         rel_ids.repeat_interleave(k_neg))

    y_true = torch.cat([torch.ones_like(pos_scores), torch.zeros_like(neg_scores)])
    y_scores = torch.cat([pos_scores, neg_scores]).detach().cpu().numpy()
    y = y_true.detach().cpu().numpy()
    auc = roc_auc_score(y, y_scores)

    preds = (y_scores > 0.5).astype(int)
    p = precision_score(y, preds, zero_division=0)
    r = recall_score(y, preds, zero_division=0)
    f1 = f1_score(y, preds, zero_division=0)
    return auc, p, r, f1

results = []
all_pos_eidx_list, all_rel_ids_list = [], []

for etype, splits in edge_type_splits.items():
    out = eval_relation(z_all, etype, splits, k_neg=1)
    if out is None: continue
    auc, p, r, f1 = out
    results.append((etype, auc, p, r, f1))
    rel_id = rel2id[etype]
    te = splits["test"]
    all_pos_eidx_list.append(te)
    all_rel_ids_list.append(torch.full((te.size(1),), rel_id, dtype=torch.long))
    print(f"{etype}: AUC={auc:.4f}, P={p:.3f}, R={r:.3f}, F1={f1:.3f}")

# Overall
if all_pos_eidx_list:
    all_pos_eidx = torch.cat(all_pos_eidx_list, dim=1).to(device)
    all_rel_ids = torch.cat(all_rel_ids_list, dim=0).to(device)
    # Simple overall negatives: corrupt tails with matching type (1 per pos)
    etypes_for_all = [relid_to_etype(int(r)) for r in all_rel_ids.tolist()]
    neg_src = all_pos_eidx[0]
    neg_dst_list = []
    for r_id, et in zip(all_rel_ids.tolist(), etypes_for_all):
        d_type = et[2]
        pool = torch.arange(type_offsets[d_type],
                            type_offsets[d_type] + len(node_type_map[d_type]),
                            device=device)
        neg_dst_list.append(pool[torch.randint(0, pool.numel(), (1,), device=device)])
    neg_dst = torch.stack(neg_dst_list).view(-1)
    neg_e = torch.stack([neg_src, neg_dst], dim=0)

    pos_scores = distmult_sigmoid_scores(z_all, model.rel_emb, all_pos_eidx, all_rel_ids)
    neg_scores = distmult_sigmoid_scores(z_all, model.rel_emb, neg_e, all_rel_ids)

    y_scores = torch.cat([pos_scores, neg_scores]).detach().cpu().numpy()
    y = np.concatenate([np.ones(pos_scores.numel()), np.zeros(neg_scores.numel())])

    overall_auc = roc_auc_score(y, y_scores)
    preds = (y_scores > 0.5).astype(int)
    overall_p = precision_score(y, preds, zero_division=0)
    overall_r = recall_score(y, preds, zero_division=0)
    overall_f1 = f1_score(y, preds, zero_division=0)

    print("\n🌍 Overall Performance (filtered-ish, typed tails, DistMult):")
    print(f"Overall AUC = {overall_auc:.4f}")
    print(f"Overall Precision = {overall_p:.3f}")
    print(f"Overall Recall = {overall_r:.3f}")
    print(f"Overall F1 Score = {overall_f1:.3f}")
else:
    print("\nNo test edges available for evaluation.")


Using device: cuda
📥 Loading NetworkX graph from pickle: /home/ubuntu/myproject/venv/merged_ofda_hetionet/combined_openfda_hetionet.pkl
✅ Loaded graph with 146759 nodes and 1401041 edges.
✅ Node types: ['anatomy', 'biological_process', 'cellular_component', 'drug', 'disease', 'gene/protein', 'molecular_function', 'pathway', 'effect/phenotype', 'unknown', 'exposure']
✅ Total edge types: 22
Homogeneous nodes: 146759, edges used for training: 1120824

🔁 Training with NeighborLoader mini-batches + AMP ...
Epoch 01 | Loss: 18.0152 | Seen pos edges: 106496
Epoch 02 | Loss: 17.8158 | Seen pos edges: 106496
Epoch 03 | Loss: 17.2474 | Seen pos edges: 106496
Epoch 04 | Loss: 16.5901 | Seen pos edges: 106496
Epoch 05 | Loss: 16.0009 | Seen pos edges: 106496
Epoch 06 | Loss: 15.4065 | Seen pos edges: 106496
Epoch 07 | Loss: 15.0668 | Seen pos edges: 106496
Epoch 08 | Loss: 14.7636 | Seen pos edges: 106496
Epoch 09 | Loss: 14.4805 | Seen pos edges: 106496
Epoch 10 | Loss: 14.3493 | Seen pos edges: 

In [3]:
# === Pickle (NetworkX) → R-GCN (NeighborLoader + AMP + batched inference) ===
# Fixes:
#  - DistMult scoring (relation-aware) in train + eval
#  - Typed negatives sampled *within the batch* (local), avoiding drop
#  - zero_division=0 to silence undefined metric warnings

import os, gc, random, pickle, gzip
from collections import defaultdict

import numpy as np
import torch
import torch.nn.functional as F
from torch import nn
from torch.cuda.amp import autocast, GradScaler
from torch_geometric.nn import RGCNConv
from torch_geometric.data import Data
from torch_geometric.loader import NeighborLoader
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score
import networkx as nx

# ----------------------------
# 0) Repro + Device
# ----------------------------
SEED = 42
random.seed(SEED); np.random.seed(SEED)
torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)
os.environ.setdefault("PYTORCH_CUDA_ALLOC_CONF", "expandable_segments:True")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gc.collect()
if device.type == "cuda":
    torch.cuda.empty_cache()
print("Using device:", device)

# ----------------------------
# 1) Load PICKLE (NetworkX) + normalize attrs
# ----------------------------
# Accepts a pickle produced via: pickle.dump(G, open(path,'wb')) where G is a NetworkX graph
pickle_path = "/home/ubuntu/myproject/venv/merged_ofda_hetionet/combined_openfda_hetionet.pkl"  # <- change if needed
assert os.path.exists(pickle_path), f"Path does not exist: {pickle_path}"
print(f"📥 Loading NetworkX graph from pickle: {pickle_path}")

open_fn = gzip.open if pickle_path.endswith(".gz") else open
with open_fn(pickle_path, "rb") as f:
    G0 = pickle.load(f)

# Ensure MultiDiGraph container
if isinstance(G0, (nx.MultiDiGraph, nx.MultiGraph, nx.Graph, nx.DiGraph)):
    G = nx.MultiDiGraph()
    G.add_nodes_from(G0.nodes(data=True))
    if isinstance(G0, (nx.MultiDiGraph, nx.MultiGraph)):
        for u, v, k, d in G0.edges(keys=True, data=True):
            G.add_edge(u, v, key=k, **(d or {}))
    else:
        for u, v, d in G0.edges(data=True):
            G.add_edge(u, v, **(d or {}))
else:
    raise TypeError(f"Pickle did not contain a NetworkX graph, got {type(G0)}")

# Normalize expected attributes
for n, data in G.nodes(data=True):
    if "node_type" not in data:
        data["node_type"] = data.get("type", data.get("category", data.get("kind", "unknown")))
for _, _, attr in G.edges(data=True):
    if "relation" not in attr:
        attr["relation"] = attr.get("display_relation", attr.get("relationship", attr.get("type", attr.get("label", "unknown"))))

print(f"✅ Loaded graph with {len(G.nodes)} nodes and {len(G.edges)} edges.")

# ----------------------------
# 2) Typed remap + splits
# ----------------------------
node_type_map = defaultdict(list)
node_id_map = {}
for nid, data in G.nodes(data=True):
    ntype = data.get("node_type", "unknown")
    idx = len(node_type_map[ntype])
    node_type_map[ntype].append(nid)
    node_id_map[nid] = (ntype, idx)

edge_type_map = defaultdict(list)  # (src_type, rel, dst_type) -> [(src_local, dst_local)]
for s, t, attr in G.edges(data=True):
    if s not in node_id_map or t not in node_id_map:
        continue
    rel = attr.get("relation", "unknown")
    s_t, s_i = node_id_map[s]
    t_t, t_i = node_id_map[t]
    edge_type_map[(s_t, rel, t_t)].append((s_i, t_i))

print(f"✅ Node types: {list(node_type_map.keys())}")
print(f"✅ Total edge types: {len(edge_type_map)}")

# Global ids per (type, local)
type_offsets, global_id_map = {}, {}
offset = 0
for ntype, nodes in node_type_map.items():
    type_offsets[ntype] = offset
    for i in range(len(nodes)):
        global_id_map[(ntype, i)] = offset + i
    offset += len(nodes)
num_nodes = offset

edge_type_keys = list(edge_type_map.keys())  # rel_id -> (src_type, rel, dst_type)
rel2id = {k: i for i, k in enumerate(edge_type_keys)}
num_relations = len(edge_type_keys)

all_src, all_dst, all_rel = [], [], []
edge_type_splits = {}

for etype, pairs in edge_type_map.items():
    s_type, _, t_type = etype
    if len(pairs) < 5:  # keep only relations with minimum support
        continue
    perm = torch.randperm(len(pairs))
    n_tr = int(0.8 * len(pairs))
    n_va = int(0.1 * len(pairs))
    tr = [pairs[i] for i in perm[:n_tr]]
    va = [pairs[i] for i in perm[n_tr:n_tr + n_va]]
    te = [pairs[i] for i in perm[n_tr + n_va:]]

    r = rel2id[etype]
    for s_l, t_l in tr:
        all_src.append(type_offsets[s_type] + s_l)
        all_dst.append(type_offsets[t_type] + t_l)
        all_rel.append(r)

    edge_type_splits[etype] = {
        "val": torch.tensor(
            [[type_offsets[s_type] + s for s, _ in va],
             [type_offsets[t_type] + t for _, t in va]], dtype=torch.long),
        "test": torch.tensor(
            [[type_offsets[s_type] + s for s, _ in te],
             [type_offsets[t_type] + t for _, t in te]], dtype=torch.long),
    }

edge_index_cpu = torch.tensor([all_src, all_dst], dtype=torch.long)
edge_type_cpu = torch.tensor(all_rel, dtype=torch.long)
print(f"Homogeneous nodes: {num_nodes}, edges used for training: {edge_index_cpu.size(1)}")

# For quick type lookup by GLOBAL id (used for local typed negatives)
type_names = sorted(node_type_map.keys())
type2id = {t: i for i, t in enumerate(type_names)}
global_node_type_id = torch.empty(num_nodes, dtype=torch.long)
for tname, nodes in node_type_map.items():
    t_id = type2id[tname]
    base = type_offsets[tname]
    n = len(nodes)
    global_node_type_id[base:base + n] = t_id

# Existing edges per relation (LOCAL to each (src_type,rel,dst_type))
existing_edges_per_rel = {etype: set(pairs) for etype, pairs in edge_type_map.items()}
def relid_to_etype(rel_id: int):
    return edge_type_keys[rel_id]

# ----------------------------
# 3) PyG Data + NeighborLoader
# ----------------------------
data_pyg = Data(num_nodes=num_nodes, edge_index=edge_index_cpu, edge_type=edge_type_cpu)

NUM_NEIGHBORS = [15, 10]
NBATCH_SIZE   = 4096
NUM_WORKERS   = 2  # quiets the OS warning

train_src_unique = torch.unique(edge_index_cpu[0])
train_loader = NeighborLoader(
    data_pyg,
    num_neighbors=NUM_NEIGHBORS,
    batch_size=NBATCH_SIZE,
    input_nodes=train_src_unique,
    shuffle=True,
    num_workers=NUM_WORKERS
)

# ----------------------------
# 4) Model (DistMult) + helpers
# ----------------------------
class RGCN(nn.Module):
    def __init__(self, num_nodes, num_relations, emb_dim=64, hidden_dim=128, out_dim=32, num_bases=30):
        super().__init__()
        self.emb = nn.Embedding(num_nodes, emb_dim)
        self.conv1 = RGCNConv(emb_dim, hidden_dim, num_relations=num_relations, num_bases=num_bases)
        self.conv2 = RGCNConv(hidden_dim, out_dim, num_relations=num_relations, num_bases=num_bases)
        self.rel_emb = nn.Embedding(num_relations, out_dim)

        nn.init.xavier_uniform_(self.emb.weight)
        nn.init.xavier_uniform_(self.rel_emb.weight)

    def forward(self, n_id, edge_index, edge_type):
        # n_id: global ids ordered as local nodes in the subgraph
        x = self.emb(n_id)
        x = self.conv1(x, edge_index, edge_type).relu_()
        x = self.conv2(x, edge_index, edge_type)
        return x  # local node embeddings

    def score(self, h, r, t):
        # DistMult (logits)
        return (h * r * t).sum(dim=-1)

def distmult_sigmoid_scores(z, rel_emb, eidx, rel_ids):
    h = z[eidx[0]]; t = z[eidx[1]]
    r = rel_emb(rel_ids)
    return (h * r * t).sum(dim=-1).sigmoid()

# ----------------------------
# 5) Local (in-batch) typed negatives
# ----------------------------
def build_local_type_pools(n_id_device):
    """Map node_type_id -> list of LOCAL indices present in this batch."""
    g_types = global_node_type_id.to(n_id_device.device, non_blocking=True)[n_id_device]
    pools = defaultdict(list)
    for local_idx, t in enumerate(g_types.tolist()):
        pools[t].append(local_idx)
    for k in list(pools.keys()):
        pools[k] = torch.tensor(pools[k], dtype=torch.long, device=n_id_device.device)
    return pools

def sample_neg_local(pos_local_eidx, rel_ids_local, n_id, pools, k_neg=1):
    """
    Typed negatives *within* the sampled subgraph.
    pos_local_eidx: (2,P) local indices
    rel_ids_local:  (P,)   relation ids (global rel ids)
    pools: dict[type_id] -> local node index tensor present in batch
    """
    if pos_local_eidx.numel() == 0:  # empty
        return torch.empty((2,0), dtype=torch.long, device=pos_local_eidx.device)

    P = pos_local_eidx.size(1)
    neg_src, neg_dst = [], []
    for i in range(P):
        s_l = int(pos_local_eidx[0, i])
        d_l = int(pos_local_eidx[1, i])
        r_id = int(rel_ids_local[i])
        s_type, _, d_type = relid_to_etype(r_id)
        s_tid = type2id[s_type]; d_tid = type2id[d_type]

        # pick from local pools — if missing (rare), fall back to uniform local sampling
        s_pool = pools.get(s_tid, None)
        d_pool = pools.get(d_tid, None)
        if s_pool is None or s_pool.numel() == 0:
            s_pool = torch.arange(n_id.size(0), device=n_id.device)
        if d_pool is None or d_pool.numel() == 0:
            d_pool = torch.arange(n_id.size(0), device=n_id.device)

        for _ in range(max(1, k_neg)):
            if random.random() < 0.5:
                # corrupt head
                new_s_l = s_pool[torch.randint(0, s_pool.numel(), (1,))].item()
                neg_src.append(new_s_l); neg_dst.append(d_l)
            else:
                # corrupt tail
                new_d_l = d_pool[torch.randint(0, d_pool.numel(), (1,))].item()
                neg_src.append(s_l); neg_dst.append(new_d_l)

    if len(neg_src) == 0:
        return torch.empty((2,0), dtype=torch.long, device=n_id.device)
    return torch.stack([torch.tensor(neg_src, device=n_id.device),
                        torch.tensor(neg_dst, device=n_id.device)], dim=0)

# ----------------------------
# 6) Train with NeighborLoader + AMP
# ----------------------------
EPOCHS = 40
LR     = 1e-3
K_NEG  = 1

model = RGCN(num_nodes, num_relations, emb_dim=64, hidden_dim=128, out_dim=32, num_bases=30).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=1e-5)
scaler = GradScaler(enabled=(device.type == "cuda"))

# Global training edges (for picking positives per batch)
src_glob_all = edge_index_cpu[0].to(device)
dst_glob_all = edge_index_cpu[1].to(device)
rels_all     = edge_type_cpu.to(device)

print("\n🔁 Training with NeighborLoader mini-batches + AMP ...")
for epoch in range(1, EPOCHS+1):
    model.train()
    epoch_loss = 0.0
    seen_pos = 0

    for batch in train_loader:
        batch = batch.to(device, non_blocking=True)
        n_id = batch.n_id  # global ids in this subgraph (device tensor)

        # global->local mapping
        global2local = -torch.ones(num_nodes, dtype=torch.long, device=device)
        global2local[n_id] = torch.arange(n_id.size(0), device=device)

        # positives inside batch
        in_src = global2local[src_glob_all]
        in_dst = global2local[dst_glob_all]
        mask = (in_src >= 0) & (in_dst >= 0)
        pos_idx = torch.nonzero(mask, as_tuple=False).view(-1)
        if pos_idx.numel() == 0:
            del batch, n_id, global2local, in_src, in_dst, mask, pos_idx
            if device.type == "cuda": torch.cuda.empty_cache()
            continue

        MAX_POS_PER_BATCH = 8192
        if pos_idx.numel() > MAX_POS_PER_BATCH:
            pos_idx = pos_idx[torch.randperm(pos_idx.numel(), device=device)[:MAX_POS_PER_BATCH]]

        pos_local = torch.stack([in_src[pos_idx], in_dst[pos_idx]], dim=0)
        rel_ids_local = rels_all[pos_idx]

        # local typed pools for negatives
        pools = build_local_type_pools(n_id)

        optimizer.zero_grad(set_to_none=True)
        with autocast(enabled=(device.type == "cuda")):
            z = model(n_id, batch.edge_index, batch.edge_type)

            # DistMult positives
            pos_h = z[pos_local[0]]; pos_t = z[pos_local[1]]
            pos_r = model.rel_emb(rel_ids_local)
            pos_logits = model.score(pos_h, pos_r, pos_t)

            # local typed negatives
            neg_local = sample_neg_local(pos_local, rel_ids_local, n_id, pools, k_neg=K_NEG)
            if neg_local.size(1) > 0:
                neg_h = z[neg_local[0]]; neg_t = z[neg_local[1]]
                # repeat relation ids for negs
                neg_r = model.rel_emb(rel_ids_local.repeat_interleave(K_NEG))
                neg_logits = model.score(neg_h, neg_r, neg_t)
                loss = (F.binary_cross_entropy_with_logits(pos_logits, torch.ones_like(pos_logits)) +
                        F.binary_cross_entropy_with_logits(neg_logits, torch.zeros_like(neg_logits)))
            else:
                loss = F.binary_cross_entropy_with_logits(pos_logits, torch.ones_like(pos_logits))

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        epoch_loss += float(loss.detach().cpu())
        seen_pos += pos_local.size(1)

        # cleanup
        del batch, n_id, global2local, in_src, in_dst, mask, pos_idx
        del pos_local, rel_ids_local, pools, z, pos_h, pos_t, pos_r, pos_logits
        if 'neg_local' in locals(): del neg_local, neg_h, neg_t, neg_r, neg_logits
        if device.type == "cuda": torch.cuda.empty_cache()

    print(f"Epoch {epoch:02d} | Loss: {epoch_loss:.4f} | Seen pos edges: {seen_pos}")

# ----------------------------
# 7) Batched inference (DistMult-ready embeddings)
# ----------------------------
print("\n🔎 Computing full-node embeddings (batched inference) ...")
model.eval()
OUT_DIM = 32
z_all = torch.empty((num_nodes, OUT_DIM), dtype=torch.float32, device=device)

INFER_BATCH = 16384
infer_loader = NeighborLoader(
    Data(num_nodes=num_nodes, edge_index=edge_index_cpu, edge_type=edge_type_cpu),
    num_neighbors=[-1, -1],
    batch_size=INFER_BATCH,
    input_nodes=torch.arange(num_nodes),
    shuffle=False,
    num_workers=NUM_WORKERS
)
with torch.no_grad():
    for batch in infer_loader:
        batch = batch.to(device, non_blocking=True)
        n_id = batch.n_id
        z_batch = model(n_id, batch.edge_index, batch.edge_type)
        z_all[n_id] = z_batch
        del batch, z_batch, n_id
        if device.type == "cuda": torch.cuda.empty_cache()

# ----------------------------
# 8) Evaluation (per relation + overall) — DistMult + zero_division=0
# ----------------------------
print("\n📊 Final evaluation per edge type (filtered negatives, DistMult):")

def eval_relation(z_all, etype, splits, k_neg=1):
    rel_id = rel2id[etype]
    pos_e = splits["test"]
    if pos_e.numel() == 0: return None
    N = pos_e.size(1)

    # Build typed negatives globally (tails only for simplicity)
    src_g = pos_e[0].to(device); dst_g = pos_e[1].to(device)
    rel_ids = torch.full((N,), rel_id, dtype=torch.long, device=device)
    s_type, _, d_type = etype
    d_pool_global = torch.arange(type_offsets[d_type],
                                 type_offsets[d_type] + len(node_type_map[d_type]),
                                 device=device)
    neg_src = src_g.repeat_interleave(k_neg)
    neg_dst = d_pool_global[torch.randint(0, d_pool_global.numel(), (N * k_neg,), device=device)]

    # Scores
    pos_scores = distmult_sigmoid_scores(z_all, model.rel_emb, pos_e.to(device), rel_ids)
    neg_scores = distmult_sigmoid_scores(z_all, model.rel_emb,
                                         torch.stack([neg_src, neg_dst]),
                                         rel_ids.repeat_interleave(k_neg))

    y_true = torch.cat([torch.ones_like(pos_scores), torch.zeros_like(neg_scores)])
    y_scores = torch.cat([pos_scores, neg_scores]).detach().cpu().numpy()
    y = y_true.detach().cpu().numpy()
    auc = roc_auc_score(y, y_scores)

    preds = (y_scores > 0.5).astype(int)
    p = precision_score(y, preds, zero_division=0)
    r = recall_score(y, preds, zero_division=0)
    f1 = f1_score(y, preds, zero_division=0)
    return auc, p, r, f1

results = []
all_pos_eidx_list, all_rel_ids_list = [], []

for etype, splits in edge_type_splits.items():
    out = eval_relation(z_all, etype, splits, k_neg=1)
    if out is None: continue
    auc, p, r, f1 = out
    results.append((etype, auc, p, r, f1))
    rel_id = rel2id[etype]
    te = splits["test"]
    all_pos_eidx_list.append(te)
    all_rel_ids_list.append(torch.full((te.size(1),), rel_id, dtype=torch.long))
    print(f"{etype}: AUC={auc:.4f}, P={p:.3f}, R={r:.3f}, F1={f1:.3f}")

# Overall
if all_pos_eidx_list:
    all_pos_eidx = torch.cat(all_pos_eidx_list, dim=1).to(device)
    all_rel_ids = torch.cat(all_rel_ids_list, dim=0).to(device)
    # Simple overall negatives: corrupt tails with matching type (1 per pos)
    etypes_for_all = [relid_to_etype(int(r)) for r in all_rel_ids.tolist()]
    neg_src = all_pos_eidx[0]
    neg_dst_list = []
    for r_id, et in zip(all_rel_ids.tolist(), etypes_for_all):
        d_type = et[2]
        pool = torch.arange(type_offsets[d_type],
                            type_offsets[d_type] + len(node_type_map[d_type]),
                            device=device)
        neg_dst_list.append(pool[torch.randint(0, pool.numel(), (1,), device=device)])
    neg_dst = torch.stack(neg_dst_list).view(-1)
    neg_e = torch.stack([neg_src, neg_dst], dim=0)

    pos_scores = distmult_sigmoid_scores(z_all, model.rel_emb, all_pos_eidx, all_rel_ids)
    neg_scores = distmult_sigmoid_scores(z_all, model.rel_emb, neg_e, all_rel_ids)

    y_scores = torch.cat([pos_scores, neg_scores]).detach().cpu().numpy()
    y = np.concatenate([np.ones(pos_scores.numel()), np.zeros(neg_scores.numel())])

    overall_auc = roc_auc_score(y, y_scores)
    preds = (y_scores > 0.5).astype(int)
    overall_p = precision_score(y, preds, zero_division=0)
    overall_r = recall_score(y, preds, zero_division=0)
    overall_f1 = f1_score(y, preds, zero_division=0)

    print("\n🌍 Overall Performance (filtered-ish, typed tails, DistMult):")
    print(f"Overall AUC = {overall_auc:.4f}")
    print(f"Overall Precision = {overall_p:.3f}")
    print(f"Overall Recall = {overall_r:.3f}")
    print(f"Overall F1 Score = {overall_f1:.3f}")
else:
    print("\nNo test edges available for evaluation.")


Using device: cuda
📥 Loading NetworkX graph from pickle: /home/ubuntu/myproject/venv/merged_ofda_hetionet/combined_openfda_hetionet.pkl
✅ Loaded graph with 146759 nodes and 1401041 edges.
✅ Node types: ['anatomy', 'biological_process', 'cellular_component', 'drug', 'disease', 'gene/protein', 'molecular_function', 'pathway', 'effect/phenotype', 'unknown', 'exposure']
✅ Total edge types: 22
Homogeneous nodes: 146759, edges used for training: 1120824

🔁 Training with NeighborLoader mini-batches + AMP ...
Epoch 01 | Loss: 18.0152 | Seen pos edges: 106496
Epoch 02 | Loss: 17.8158 | Seen pos edges: 106496
Epoch 03 | Loss: 17.2473 | Seen pos edges: 106496
Epoch 04 | Loss: 16.5903 | Seen pos edges: 106496
Epoch 05 | Loss: 16.0013 | Seen pos edges: 106496
Epoch 06 | Loss: 15.4070 | Seen pos edges: 106496
Epoch 07 | Loss: 15.0670 | Seen pos edges: 106496
Epoch 08 | Loss: 14.7634 | Seen pos edges: 106496
Epoch 09 | Loss: 14.4800 | Seen pos edges: 106496
Epoch 10 | Loss: 14.3489 | Seen pos edges: 